In [6]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import requests
import re

my_id = 'realstone.in'
org = '400201' # 대법원
JO = '민법' #%EB%AF%BC%EB%B2%95
base_url = f'https://www.law.go.kr/DRF/lawSearch.do?OC={my_id}&target=prec&type=XML&display=100'
event_base_url = 'http://www.law.go.kr'

'''
https://www.law.go.kr/DRF/lawSearch.do?OC=realstone.in&target=prec&type=XML&display=100&page=1&search=3&org=400201&JO=%EB%AF%BC%EB%B2%95
'''

# 사건명 사건번호 사건종류명 판례일련번호 판례상세링크 법원명
event_Title = []
event_num = []
event_type = []
jPrecedent_num = []
jPrecedent_link_get = []
jPrecedent_link_repl = []
court_name = []
court_num = []
court_detail = []
court_reference = []
court_Decision = []
page=1
url = f'{base_url}&page={page}&org={org}&JO={JO}'
#&org={org}
response = requests.get(url)
title_XML = response.text
title_soup = BeautifulSoup(title_XML, 'xml') 

total_page = int(title_soup.select('totalCnt')[0].string)//100+1

for page in tqdm(range(1, total_page+1)) :
    url = f'{base_url}&page={page}&org={org}&JO={JO}'
    response = requests.get(url)
    title_XML = response.text
    title_soup = BeautifulSoup(title_XML, 'xml') 

    for i in range(len(title_soup.select('사건명'))) :
        event_Title.append(title_soup.select('사건명')[i].string)
        event_num.append(title_soup.select('사건번호')[i].string)
        event_type.append(title_soup.select('사건종류명')[i].string)
        jPrecedent_num.append(title_soup.select('판례일련번호')[i].string)
        jPrecedent_link_get.append(title_soup.select('판례상세링크')[i].string)
        court_name.append(title_soup.select('법원명')[i].string)
        

for i in tqdm(range(len(jPrecedent_link_get))) :
    jPrecedent_link_repl.append(jPrecedent_link_get[i][:-10].replace('HTML', 'XML'))
    event_url = f'{event_base_url}{jPrecedent_link_repl[i]}'
    event_response = requests.get(event_url)
    main_XMLevent = event_response.text
    main_soup = BeautifulSoup(main_XMLevent, 'xml')
    #print(event_url)
    court_num.append(main_soup.select('사건번호')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_detail.append(main_soup.select('판례내용')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_reference.append(main_soup.select('참조조문')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_Decision.append(main_soup.select('판시사항')[0].string.replace('</br>', '').replace('<br/>', ''))


  0%|          | 0/24 [00:00<?, ?it/s]

  0%|          | 0/1824 [00:00<?, ?it/s]

In [8]:
import pandas as pd
'''
event_Title 
event_num 
event_type 
jPrecedent_num 
jPrecedent_link_get 
court_name 
court_num 
court_detail 
court_reference 
court_Decision
'''

df = pd.DataFrame({'event_Title':event_Title, 'event_num':event_num, 'event_type':event_type, 'jPrecedent_num':jPrecedent_num, \
                  'jPrecedent_link_get':jPrecedent_link_get, 'court_name':court_name, 'court_num':court_num, \
                   'court_detail':court_detail, 'court_reference':court_reference, 'court_Decision':court_Decision})

df.to_csv('court.csv',encoding='cp949')

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import pandas as pd
import pickle

df = pd.read_csv('court.csv', encoding='cp949')
# 줄거리 텍스트 데이터 전처리
okt = Okt()
text = df['court_detail'].str.replace("\n", '')

# 텍스트를 형태소 단위로 나누기
processed_text = [' '.join(okt.morphs(sentence)) for sentence in text]

# TF-IDF 벡터라이저 초기화 및 학습
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_text)

# 모델과 TF-IDF 매트릭스 저장
with open('court_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

with open('court_tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

# 전처리된 텍스트 저장
with open('court_processed_text.pkl', 'wb') as f:
    pickle.dump(processed_text, f)

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import pandas as pd
import pickle


df = pd.read_csv('court.csv', encoding='cp949')

# 저장된 모델과 매트릭스 불러오기
with open('court_tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('court_tfidf_matrix.pkl', 'rb') as f:
    tfidf_matrix = pickle.load(f)

with open('court_processed_text.pkl', 'rb') as f:
    processed_text = pickle.load(f)

def recommend_court(input_sentence, top_n=5):
    # 입력 문장을 Okt를 사용하여 전처리
    okt = Okt()
    processed_input = ' '.join(okt.morphs(input_sentence))
    
    # 입력 문장을 TF-IDF 변환
    input_tfidf = tfidf_vectorizer.transform([processed_input])
    
    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(input_tfidf, tfidf_matrix).flatten()
    
    # 유사도 순으로 정렬된 상위 n개 인덱스 추출
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # 유사도 0인 경우 처리
    recommendations = []
    for idx in top_indices:
        if cosine_similarities[idx] == 0:
            break
        recommendations.append((df.iloc[idx]['event_Title'], \
                                df.iloc[idx]['event_num'], \
                                df.iloc[idx]['event_type'], \
                                df.iloc[idx]['jPrecedent_num'], \
                                df.iloc[idx]['court_name'], \
                                df.iloc[idx]['court_num'], \
                                df.iloc[idx]['court_detail'], \
                                df.iloc[idx]['court_reference'], \
                                df.iloc[idx]['court_Decision'], \
                                cosine_similarities[idx]))
    
    if len(recommendations) == 0:
        return "유사한 판례를 찾을 수 없습니다."
    else:
        return recommendations

court_input = str(input('상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :'))
court_detail_OX = str(input('판례 및 판시 사항을 보시겠습니까?(Y/N) : '))
input_sentence = court_input

# 판례
recommended_court = recommend_court(input_sentence)
if isinstance(recommended_court, str):
    print(recommended_court)
else:
    for event_Title, event_num, event_type, jPrecedent_num, court_name, court_num, court_detail, court_reference, court_Decision, similarity  \
    in recommended_court:
        
        if (court_detail_OX == 'Y') or (court_detail_OX == 'y') :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f})\n사건 번호 : {event_num} \n사건 종류 : {event_type}\
                    \n판례 일련번호 : {jPrecedent_num} \
                    \n법원 : {court_name} \n 사건 번호 : {court_num} \n\n 판례 내용 :\n{court_detail} \n\n 참조조문 \n{court_reference} \
                    \n\n판시사항 :\n{court_Decision}\n\n\n")
            
        elif (court_detail_OX == 'N') or (court_detail_OX == 'n') :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f})\n사건 번호 : {event_num} \n사건 종류 : {event_type}\
                    \n판례 일련번호 : {jPrecedent_num} \
                    \n법원 : {court_name} \n 사건 번호 : {court_num} \n\n 참조조문 \n{court_reference}\n\n\n")


상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :
부동산 전세사기를 당했는데 건물주가 도망갔어
판례 및 판시 사항을 보시겠습니까?(Y/N) : 
Y
사건 명 : 손해배상 (유사도: 0.17)
사건 번호 : 2013다11621 
사건 종류 : 민사                    
판례 일련번호 : 178071                     
법원 : 대법원 
 사건 번호 : 2013다11621 

 판례 내용 :
【원고, 상고인 겸 부대피상고인】 【피고, 피상고인 겸 부대상고인】 주식회사 코리아본뱅크 (Korea Bone Bank CO., LTD) [변경 후 상호: 주식회사 셀루메드 (CELLUMED CO., LTD)] (소송대리인 법무법인 율우 담당변호사 이창헌 외 1인)【피고, 피상고인】 【원심판결】 서울고법 2013. 1. 10. 선고 2010나84682 판결【주    문】  상고와 부대상고를 모두 기각한다.  원고와 피고 주식회사 코리아본뱅크 사이의 상고비용과 부대상고비용은 각자 부담하고, 원고와 피고 2, 피고 3 사이의 상고비용은 원고가 부담한다.【이    유】  상고이유와 부대상고이유를 함께 판단한다.  1. 피고 주식회사 코리아본뱅크의 부대상고이유 제1, 2, 4, 6점에 대하여  원심은 채택 증거를 종합하여 그 판시와 같은 사실을 인정한 다음, 피고 회사의 대표이사인 피고 2는 피고 회사의 레이저기술 개발 등과 관련하여 허위의 사실을 공시하거나 유포하였고, 원고는 피고 회사의 이 사건 유상증자에 참여하여 정상주가보다 높은 가격에 신주를 인수하고 허위사실 유포 등에 의하여 비정상적으로 높게 형성된 주가를 정상주가로 오인하여 코스닥시장에서 피고 회사의 주식을 매수하여 손해를 입었으며, 피고 3은 직무 수행 의사 없이 피고 회사에 이사 명의를 빌려주어 피고 2의 위와 같은 행위를 방치하였으므로, 피고들은 연대하여 원고가 이 사건 주식을 취득함으로써 입은 손해를 배상할 책임이 있다고 판단하였다.  원심판결 이유를 관련 법리와 

soup

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from konlpy.tag import Okt
import pandas as pd
import pickle  # 모델 저장 및 로딩을 위해
import numpy as np
import nltk

def model(filepath, court_detail):
    tfidf = TfidfVectorizer(tokenizer=Okt().morphs, max_features=500)
    tfidf.fit(court_detail)
    # 모델 저장
    with open(filepath, 'wb') as f:
        pickle.dump(tfidf, f)
'''
TfidfVectorizer() 하이퍼 파라미터
    min_df : 설정한 값보다 특정 Token의 df 값이 더 적게 나오면 벡터화 과정에서 제거
    anlayzer : 분석 단위를 의미, ‘word’의 경우 간어 하나를 단위로, ‘char’는 문자 하나를 단위로
    sublinear_tf : 문서의 단어 빈도수(tf:term frequency)에 대한 smoothing 여부를 설정
    ngram_range : 빈도의 기본 단위를 어떤 범위의 n-gram으로 설정할 것인지를 보는 인자
    max_features : 각 벡터의 최대 길이(특징의 길이)를 설정
'''
        
        
# 저장된 TF-IDF 모델을 불러와 사용하는 함수
def load_tfidf_model(filepath='tfidf_model.pkl'):
    with open(filepath, 'rb') as f:
        return pickle.load(f)
    
def model_with_pretrained_tfidf(detail, num, court_detail):
    tfidf = load_tfidf_model()  # 저장된 모델 불러오기
    all_detail = court_detail + [detail]
    tfidf_matrix = tfidf.transform(all_detail)  # fit_transform 대신 transform 사용
    cosine_sim = cosine_similarity(tfidf_matrix[-1], tfidf_matrix[:-1])
    cosine_similar_indices = np.argsort(cosine_sim)[-num:][::-1]
    return cosine_similar_indices

court_detail = pd.read_csv('court.csv', encoding='cp949')['court_detail']
path = 'tfidf_model.pkl'
model(path, court_detail)

'''
detail = str(input('내용을 입력해 주세요 : '))#'새로 선임된 청산인을 기존 이사회가 해임시킬 수 있어?'
num = int(input('몇 개의 판례를 보시겠습니까? (최대 5개) : '))
if num == 1:
    OX = str(input('판례 내용을 보시겠습니까?(O, X) : '))
    print(f'질문 : "{detail}"')
    index = model_with_pretrained_tfidf(detail, num, court_detail)
    print(f'사건 번호 : {court_num[index[0][0]]}')
    print(f'참조 조문 : {court_reference[index[0][0]]}')
    print(f'판시 사항 : {court_Decision[index[0][0]]}')
    if OX == 'O' :
        print(f'유사한 판례 : {court_detail[index[0][0]]}')
elif num <= 5:
    OX = str(input('판례 내용을 보시겠습니까?(O, X) : '))
    print(f'질문 : "{detail}"')
    index = model_with_pretrained_tfidf(detail, num, court_detail)
    for i in index[0][:num]:
        print(f'사건 번호 : {court_num[index[0][i]]}')
        print(f'참조 조문 : {court_reference[index[0][i]]}')
        print(f'판시 사항 : {court_Decision[index[0][i]]}')
        if OX == 'O' :
            print(f'유사한 판례 : {court_detail[index[0][i]]}')
elif num > 5:
    OX = str(input('판례 내용을 보시겠습니까?(O, X) : '))
    print(f'질문 : "{detail}"')
    index = model_with_pretrained_tfidf(detail, 5, court_detail)
    for i in index[0][:5] :
        print(f'사건 번호 : {court_num[index[0][i]]}')
        print(f'참조 조문 : {court_reference[index[0][i]]}')
        print(f'판시 사항 : {court_Decision[index[0][i]]}')
        if OX == 'O' :
            print(f'유사한 판례 : {court_detail[index[0][i]]}')
'''

C:\Users\reals\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


KeyboardInterrupt: 

In [ ]:
import pandas as pd
import xml.etree.ElementTree as ET
from urllib.request import urlopen
from tqdm import trange
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import requests
import re

my_id = 'realstone.in'
org = '400201' # 대법원
JO = '근로' #%EB%AF%BC%EB%B2%95
base_url = f'https://www.law.go.kr/DRF/lawSearch.do?OC={my_id}&target=prec&type=XML&display=100'
event_base_url = 'http://www.law.go.kr'

'''
https://www.law.go.kr/DRF/lawSearch.do?OC=realstone.in&target=prec&type=XML&display=100&page=1&search=3&org=400201&JO=%EB%AF%BC%EB%B2%95
'''

# 사건명 사건번호 사건종류명 판례일련번호 판례상세링크 법원명
event_Title = []
event_num = []
event_type = []
jPrecedent_num = []
jPrecedent_link_get = []
jPrecedent_link_repl = []
court_name = []
court_num = []
court_detail = []
court_reference = []
court_Decision = []
page=1
url = f'{base_url}&page={page}&JO={JO}'
#&org={org}
response = requests.get(url)
title_XML = response.text
title_soup = BeautifulSoup(title_XML, 'xml') 

total_page = int(title_soup.select('totalCnt')[0].string)//100+1

for page in tqdm(range(1, total_page+1)) :
    url = f'{base_url}&page={page}&org={org}&JO={JO}'
    response = requests.get(url)
    title_XML = response.text
    title_soup = BeautifulSoup(title_XML, 'xml') 

    for i in range(len(title_soup.select('사건명'))) :
        event_Title.append(title_soup.select('사건명')[i].string)
        event_num.append(title_soup.select('사건번호')[i].string)
        event_type.append(title_soup.select('사건종류명')[i].string)
        jPrecedent_num.append(title_soup.select('판례일련번호')[i].string)
        jPrecedent_link_get.append(title_soup.select('판례상세링크')[i].string)
        court_name.append(title_soup.select('법원명')[i].string)
        

for i in tqdm(range(len(jPrecedent_link_get))) :
    jPrecedent_link_repl.append(jPrecedent_link_get[i][:-10].replace('HTML', 'XML'))
    event_url = f'{event_base_url}{jPrecedent_link_repl[i]}'
    event_response = requests.get(event_url)
    main_XMLevent = event_response.text
    main_soup = BeautifulSoup(main_XMLevent, 'xml')
    #print(event_url)
    court_num.append(main_soup.select('사건번호')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_detail.append(main_soup.select('판례내용')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_reference.append(main_soup.select('참조조문')[0].string.replace('</br>', '').replace('<br/>', ''))
    court_Decision.append(main_soup.select('판시사항')[0].string.replace('</br>', '').replace('<br/>', ''))


In [ ]:
import pandas as pd
'''
event_Title 
event_num 
event_type 
jPrecedent_num 
jPrecedent_link_get 
court_name 
court_num 
court_detail 
court_reference 
court_Decision
'''

df = pd.DataFrame({'event_Title':event_Title, 'event_num':event_num, 'event_type':event_type, 'jPrecedent_num':jPrecedent_num, \
                  'jPrecedent_link_get':jPrecedent_link_get, 'court_name':court_name, 'court_num':court_num, \
                   'court_detail':court_detail, 'court_reference':court_reference, 'court_Decision':court_Decision})

df.to_csv('court_work.csv',encoding='cp949')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from konlpy.tag import Okt
import pandas as pd
import pickle

df = pd.read_csv('court_work.csv', encoding='cp949')
# 줄거리 텍스트 데이터 전처리
okt = Okt()
text = df['court_detail'].str.replace("\n", '')

# 텍스트를 형태소 단위로 나누기
processed_text = [' '.join(okt.morphs(sentence)) for sentence in text]

# TF-IDF 벡터라이저 초기화 및 학습
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_text)

# 모델과 TF-IDF 매트릭스 저장
with open('court_tfidf_vectorizer.pkl', 'wb') as f:
    pickle.dump(tfidf_vectorizer, f)

with open('court_tfidf_matrix.pkl', 'wb') as f:
    pickle.dump(tfidf_matrix, f)

# 전처리된 텍스트 저장
with open('court_processed_text.pkl', 'wb') as f:
    pickle.dump(processed_text, f)

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from konlpy.tag import Okt
import pandas as pd
import pickle

# 영화 데이터 로드
df = pd.read_csv('court_work.csv', encoding='cp949')

# 저장된 모델과 매트릭스 불러오기
with open('court_tfidf_vectorizer.pkl', 'rb') as f:
    tfidf_vectorizer = pickle.load(f)

with open('court_tfidf_matrix.pkl', 'rb') as f:
    tfidf_matrix = pickle.load(f)

with open('court_processed_text.pkl', 'rb') as f:
    processed_text = pickle.load(f)

def recommend_court(input_sentence, top_n=5):
    # 입력 문장을 Okt를 사용하여 전처리
    okt = Okt()
    processed_input = ' '.join(okt.morphs(input_sentence))
    
    # 입력 문장을 TF-IDF 변환
    input_tfidf = tfidf_vectorizer.transform([processed_input])
    
    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(input_tfidf, tfidf_matrix).flatten()
    
    # 유사도 순으로 정렬된 상위 n개 인덱스 추출
    top_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    # 유사도 0인 경우 처리
    recommendations = []
    for idx in top_indices:
        if cosine_similarities[idx] == 0:
            break
        recommendations.append((df.iloc[idx]['event_Title'], \
                                df.iloc[idx]['event_num'], \
                                df.iloc[idx]['event_type'], \
                                df.iloc[idx]['jPrecedent_num'], \
                                df.iloc[idx]['court_name'], \
                                df.iloc[idx]['court_num'], \
                                df.iloc[idx]['court_detail'], \
                                df.iloc[idx]['court_reference'], \
                                df.iloc[idx]['court_Decision'], \
                                cosine_similarities[idx]))
    
    if len(recommendations) == 0:
        return "유사한 판례를 찾을 수 없습니다."
    else:
        return recommendations

'''
event_Title 
event_num 
event_type 
jPrecedent_num 
court_name 
court_num 
court_detail 
court_reference 
court_Decision
'''

print('상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :')
court_input = str(input())
print('판례 및 판시 사항을 보시겠습니까?(Y/N) : ')
court_detail_OX = str(input())
input_sentence = court_input

# 판례
recommended_court = recommend_court(input_sentence)
if isinstance(recommended_court, str):
    print(recommended_court)
else:
    for event_Title, event_num, event_type, jPrecedent_num, court_name, court_num, court_detail, court_reference, court_Decision, similarity  in recommended_court:
        if (court_detail_OX == 'Y') or (court_detail_OX == 'y') :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f})\n사건 번호 : {event_num} \n사건 종류 : {event_type} \n판례 일련번호 : {jPrecedent_num} \
            \n법원 : {court_name} \n 사건 번호 : {court_num} \n\n 판례 내용 :\n{court_detail} \n\n 참조조문 \n{court_reference} \
            \n\n판시사항 :\n{court_Decision}\n\n\n")
        elif (court_detail_OX == 'N') or (court_detail_OX == 'n') :
            print(f"사건 명 : {event_Title} (유사도: {similarity:.2f})\n사건 번호 : {event_num} \n사건 종류 : {event_type} \n판례 일련번호 : {jPrecedent_num} \
            \n법원 : {court_name} \n 사건 번호 : {court_num} \n\n 참조조문 \n{court_reference}\n\n\n")


상황을 작성해 주세요 (자세히 작성 시 검색에 도움이 됩니다.) :
회사에서 월급을 안 주고 버티고 있어
판례 및 판시 사항을 보시겠습니까?(Y/N) : 
Y
사건 명 : 임금등 (유사도: 0.23)
사건 번호 : 93다32514 
사건 종류 : 민사 
판례 일련번호 : 110896             
법원 : 대법원 
 사건 번호 : 93다32514 

 판례 내용 :
【원고, 피상고인】   우종용 외 23인【피고, 상고인】   엠코.에이.앤드.이, 인크               소송대리인 법무법인 한미합동법률사무소 외 1인【원심판결】 서울고등법원 1993.5.20. 선고 92나25221 판결【주    문】  상고를 기각한다. 상고비용은 피고의 부담으로 한다. 【이    유】  상고이유를 본다.   제1점에 대하여  원심이 취사한 증거를 기록에 비추어 보면 원심의 사실인정을 수긍할 수 있고, 원심이 피고의 근로자는 당초에는 평일에 8시간, 토요일에는 4시간씩 근무하여 오던 중 근로자들의 발의로 토요일을 휴무로 하는 대신 월요일부터 목요일까지는 9시간씩 금요일에는 8시간씩 근무하여 오다가, 1989년부터는 월요일부터 금요일까지 매일 8시간씩 주 40시간을 근무하여 온 사실, 피고는 원고들별로 각 시간급 임금을 책정하여 그 임금에 원고들이 매월 실제 근로한 시간 수를 곱한(시간외 근로를 한 경우에는 소정의 비율을 가산한 시간수를 곱한) 금액을 월급으로 지급하여 왔고, 피고의 내규로 한국 신정, 구정, 미국 대통령의 날 등 13개를 유급 공휴일로 정하였으나, 주휴일인 매주 토요일과 일요일은 무급으로 실시하여 왔으며, 휴가는 1개월에 3.33시간이 생기나 고용연도에 대해 완전히 생기기 전에는 미리 휴가를 사용할 수 없도록 규정하고 있는 사실 등을 인정하고, 원고들에게 매월 정액의 임금을 지급하여 왔다는 피고의 주장을 인정하지 아니한 조처도 수긍이 가고, 거기에 채증법칙을 어긴 위법이 있다고 할 수 없다.   을 제3호증의 1 내지 24(고용계약서)가 
     

In [19]:
court_reference

'근로기준법 제28조, 민법 제108조 제1항'